In [1]:
import requests
import pandas as pd
import csv
from bs4 import BeautifulSoup

In [4]:
url = "https://www.cazoo.co.uk/cars/"
response = requests.get(url)
response.status_code

200

In [9]:
soup = BeautifulSoup(response.text)
soup.find_all('p', class_="full-pricestyles__Price-sc-12l0fhp-0 kZsXtM")

[<p class="full-pricestyles__Price-sc-12l0fhp-0 kZsXtM" data-test-id="card-pricing-full-price-gb">£6,500</p>,
 <p class="full-pricestyles__Price-sc-12l0fhp-0 kZsXtM" data-test-id="card-pricing-full-price-gb">£6,550</p>,
 <p class="full-pricestyles__Price-sc-12l0fhp-0 kZsXtM" data-test-id="card-pricing-full-price-gb">£6,700</p>,
 <p class="full-pricestyles__Price-sc-12l0fhp-0 kZsXtM" data-test-id="card-pricing-full-price-gb">£7,000</p>,
 <p class="full-pricestyles__Price-sc-12l0fhp-0 kZsXtM" data-test-id="card-pricing-full-price-gb">£7,000</p>,
 <p class="full-pricestyles__Price-sc-12l0fhp-0 kZsXtM" data-test-id="card-pricing-full-price-gb">£7,000</p>,
 <p class="full-pricestyles__Price-sc-12l0fhp-0 kZsXtM" data-test-id="card-pricing-full-price-gb">£7,000</p>,
 <p class="full-pricestyles__Price-sc-12l0fhp-0 kZsXtM" data-test-id="card-pricing-full-price-gb">£7,250</p>,
 <p class="full-pricestyles__Price-sc-12l0fhp-0 kZsXtM" data-test-id="card-pricing-full-price-gb">£7,250</p>,
 <p class=